# Training - Other Stuff

In [1]:
!pip3 install --upgrade openai -q

In [64]:
import json, openai, time, pandas, random, getpass
from openai import cli
from types import SimpleNamespace
from sklearn.model_selection import train_test_split
from file_read_backwards import FileReadBackwards

work_dir = "/Users/eric.pinzur/Documents/slackbot2000"
openai.api_key = getpass.getpass(prompt="Please enter your OpenAI API Key")

### List Fine-Tunes

In [72]:
import pprint, datetime

result = openai.FineTune.list()

for fine_tune in result["data"]:
    item = fine_tune.to_dict()
    del item["object"]
    del item["training_files"]
    del item["validation_files"]
    del item["organization_id"]
    del item["result_files"]
    item["hyperparams"] = item["hyperparams"].to_dict()
    item["created_at"] = datetime.datetime.fromtimestamp(item["created_at"]).isoformat()
    item["updated_at"] = datetime.datetime.fromtimestamp(item["updated_at"]).isoformat()
    pprint.pprint(item, sort_dicts=False)
    print()

{'id': 'ft-dtMxYnYOF07E8LOhZcyzStlU',
 'hyperparams': {'n_epochs': 4,
                 'batch_size': 1,
                 'prompt_loss_weight': 0.01,
                 'learning_rate_multiplier': 0.1},
 'model': 'ada',
 'created_at': '2023-05-03T15:35:35',
 'updated_at': '2023-05-03T15:50:28',
 'status': 'succeeded',
 'fine_tuned_model': 'ada:ft-personal-2023-05-03-13-50-27'}

{'id': 'ft-ozxFO5z1gOLAuHw5n6NIp9hP',
 'hyperparams': {'n_epochs': 10,
                 'batch_size': None,
                 'prompt_loss_weight': 0.01,
                 'learning_rate_multiplier': None},
 'model': 'davinci',
 'created_at': '2023-08-04T21:57:32',
 'updated_at': '2023-08-05T00:35:45',
 'status': 'failed',
 'fine_tuned_model': None}

{'id': 'ft-RGKqXw7u6A5xZjwI6z3oVvrj',
 'hyperparams': {'n_epochs': 2,
                 'batch_size': None,
                 'prompt_loss_weight': 0.01,
                 'learning_rate_multiplier': None},
 'model': 'davinci',
 'created_at': '2023-08-05T15:20:22',
 'update

In [74]:
openai.FineTune.retrieve("ft-JXAXQtHd74y6fIDmeQyti3ZP")

<FineTune fine-tune id=ft-JXAXQtHd74y6fIDmeQyti3ZP at 0x127fad950> JSON: {
  "object": "fine-tune",
  "id": "ft-JXAXQtHd74y6fIDmeQyti3ZP",
  "hyperparams": {
    "n_epochs": 2,
    "batch_size": 4,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": 0.02
  },
  "organization_id": "org-CUf0quJkjZ1JS09kPR5dqcXn",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-SRadZhsiDk2ioHlNvIIRk7Ye",
      "purpose": "fine-tune",
      "filename": "/Users/eric.pinzur/Documents/slackbot2000/conversation_user_stripped_examples_prepared_train.jsonl",
      "bytes": 2105562,
      "created_at": 1691677593,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [
    {
      "object": "file",
      "id": "file-osYJgqzlIvdXUF9bevS5gxrw",
      "purpose": "fine-tune-results",
      "filename": "compiled_results.csv",
      "bytes": 97187,
      "created_at": 1691729397,
      "status": "processed

### Cancel Fine-Tunes

In [18]:
openai.FineTune.cancel("ft-TUEyMRXlB2es1JaeILlCRH7i")

InvalidRequestError: Cannot cancel a job ft-TUEyMRXlB2es1JaeILlCRH7i that already has status "failed".

### Delete Training Files

#### Delete training Files for all finished fine-tunes

In [19]:
result = openai.FineTune.list()

def delete_files(files):
    for file in files:
        if file["status"] == "processed":
            openai.File.delete(file["id"])

for fine_tune in result["data"]:
    if fine_tune["status"] in ["failed", "succeeded"]:
        delete_files(fine_tune["training_files"])
        delete_files(fine_tune["validation_files"])

### Delete FineTune Models

### Other stuff

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

file1 = open('next_message_user.jsonl', 'r')
X = []
y = []

while True:
    line = file1.readline()

    if not line:
        break

    data = json.loads(line)
    X.append(data["prompt"])
    y.append(data["completion"])

file1.close()

# Let's assume you have your data in `X` (features) and `y` (labels)
# Split data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Since you have multilabels, convert them into a binary form
mlb = MultiLabelBinarizer()
y_test_transformed = mlb.fit_transform(y_test)
y_pred_transformed = mlb.transform(y_pred)

# Calculate the F1 score
f1 = f1_score(y_test_transformed, y_pred_transformed, average='macro')  # Or 'micro', 'weighted' based on need

print(f"F1 Score: {f1}")


In [ ]:
!pip3 install -U scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

In [ ]:
X

In [ ]:
y

In [5]:
print('Checking for existing uploaded files.')
results = []

# Get the complete list of uploaded files in our subscription.
files = openai.File.list().data
print(f'Found {len(files)} total uploaded files in the subscription.')

# Enumerate all uploaded files, extracting the file IDs for the
# files with file names that match your training dataset file and
# validation dataset file names.
for item in files:
    if item["filename"] in [training_file_name, validation_file_name]:
        results.append(item["id"])
print(f'Found {len(results)} already uploaded files that match our files')

# Enumerate the file IDs for our files and delete each file.
print(f'Deleting already uploaded files.')
for id in results:
    openai.File.delete(sid = id)

Checking for existing uploaded files.
Found 2 total uploaded files in the subscription.
Found 2 already uploaded files that match our files
Deleting already uploaded files.
